# example.py in HODLR github

# https://github.com/SAFRAN-LAB/HODLR/blob/master/python/example/example.py

In [1]:
import numpy as np
import pyhodlrlib

# Size of the matrix:
N = 1000
x = np.sort(np.random.rand(N))
print(x.shape)
# Size of leaf level:
M = 200

# Returning the Gaussian Kernel:
class Kernel(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
        if(i == j):
            return 10
        else:
            return np.exp(-(x[i] - x[j])**2)

K = Kernel(N)
# What we are doing here is explicitly generating 
# the matrix from its entries
A = K.getMatrix(0, 0, N, N)
print(A.shape)
# Tolerance for all factorizations:
eps = 1e-12

# If we are assembling a symmetric matrix:
is_sym = True
# If we know that the matrix is also PD:
# By setting the matrix to be symmetric-positive definite, 
# we trigger the fast symmetric factorization method to be used
# In all other cases the fast factorization method is used
is_pd = True
# Creating the HODLR object:
T = pyhodlrlib.HODLR(N, M, eps)
T.assemble(K, 'rookPivoting', is_sym, is_pd)

# Random vector to take product with:
x = np.random.rand(N)
# Finding b using HODLR:
b_hodlr = T.matmatProduct(x)
# Finding b using direct MatVec:
b = A @ x
# Verifying the accuracy of the MatMat:
print('Error in Matrix-Matrix Multiplication:', np.linalg.norm(b_hodlr.ravel() - b.ravel()) / np.linalg.norm(b))

# Factorize elements of the tree:
T.factorize()
# Solving for x in A x = b:
x_hodlr = T.solve(b)
print(x_hodlr.shape,b.shape)
# Computing the relative error:
print('Error in Solve:', np.linalg.norm(x_hodlr.ravel() - x) /  np.linalg.norm(x))

# Finding log determinant:
logdet_hodlr = T.logDeterminant()
# Finding logdet:
logdet = 2 * np.sum(np.log(abs(np.diag(np.linalg.cholesky(A)))))
print('Error in Log-Determinant Computation:', abs(logdet_hodlr - logdet))

# When system described is SPD:
if(is_sym and is_pd):
    # Setting y = W^T x
    y = T.symmetricFactorTransposeProduct(x)
    # Getting b = W (W^T x)
    b_hodlr = T.symmetricFactorProduct(y)
    print('Error in Symmetric Factor Multiplications:', np.linalg.norm(b_hodlr.ravel() - b.ravel()) / np.linalg.norm(b))

    # Directly obtaining the symmetric factor matrix:
    W = T.getSymmetricFactor()
    print('Error in Getting Symmetric Factor:', np.mean(abs(W @ W.T - A)))

(1000,)
(1000, 1000)
Error in Matrix-Matrix Multiplication: 2.5428764967620053e-14
(1000, 1) (1000,)
Error in Solve: 2.0762204025187137e-12
Error in Log-Determinant Computation: 9.094947017729282e-13
Error in Symmetric Factor Multiplications: 2.5366937452397554e-14
Error in Getting Symmetric Factor: 2.6573587663669685e-14


# Code https://peterroelants.github.io/posts/gaussian-process-tutorial/

In [2]:

import scipy
from scipy.spatial import distance as dist

In [3]:
# Define the exponentiated quadratic 
def exponentiated_quadratic(xa, xb):
    """Exponentiated quadratic  with σ=1"""
    # L2 distance (Squared Euclidian)
    sq_norm = -0.5 * scipy.spatial.distance.cdist(xa, xb, 'sqeuclidean')
    return np.exp(sq_norm)

In [4]:
# Gaussian process posterior
def GP(X1, y1, X2, kernel_func):
    """
    Calculate the posterior mean and covariance matrix for y2
    based on the corresponding input X2, the observations (y1, X1), 
    and the prior kernel function.
    """
    # Kernel of the observations
    Σ11 = kernel_func(X1, X1)
    # Kernel of observations vs to-predict
    Σ12 = kernel_func(X1, X2)
    # Solve
    solved = scipy.linalg.solve(Σ11, Σ12, assume_a='pos').T
    # Compute posterior mean
    μ2 = solved @ y1
    # Compute the posterior covariance
    Σ22 = kernel_func(X2, X2)
    Σ2 = Σ22 - (solved @ Σ12)
    return μ2, Σ2  # mean, covariance

In [5]:
# Compute the posterior mean and covariance

# Define the true function that we want to regress on
f_sin = lambda x: (np.sin(x)).flatten()

n1 = 8  # Number of points to condition on (training points)
n2 = 75  # Number of points in posterior (test points)
ny = 5  # Number of functions that will be sampled from the posterior
domain = (-6, 6)

# Sample observations (X1, y1) on the function
X1 = np.random.uniform(domain[0]+2, domain[1]-2, size=(n1, 1))
y1 = f_sin(X1)
# Predict points at uniform spacing to capture function
X2 = np.linspace(domain[0], domain[1], n2).reshape(-1, 1)
# Compute posterior mean and covariance
μ2, Σ2 = GP(X1, y1, X2, exponentiated_quadratic)
# Compute the standard deviation at the test points to be plotted
σ2 = np.sqrt(np.diag(Σ2))

# Draw some samples of the posterior
y2 = np.random.multivariate_normal(mean=μ2, cov=Σ2, size=1)

# code gpr by changing the function getMatrixEntry

In [6]:
import scipy
from scipy.spatial import distance as dist
class Kernel(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, X, Y):
        
        sq=-0.5*dist.cdist(X, Y, 'sqeuclidean')
        return np.exp(sq)
    def GP_hod(self,X1, y1, X2):
        """
        Calculate the posterior mean and covariance matrix for y2
        based on the corresponding input X2, the observations (y1, X1), 
        and the prior kernel function.
        """
        K = Kernel(N1)
        # What we are doing here is explicitly generating 
        # the matrix from its entries
        A11=K.getMatrixEntry(X1,X1)
        #print(A11,"A11")
       
        # Tolerance for all factorizations:
        eps = 1e-12

        # If we are assembling a symmetric matrix:
        is_sym = True
        # If we know that the matrix is also PD:
        # By setting the matrix to be symmetric-positive definite, 
        # we trigger the fast symmetric factorization method to be used
        # In all other cases the fast factorization method is used
        is_pd = True
        
        # Creating the HODLR object:
        
        #M=3
        #T = pyhodlrlib.HODLR(N1, M, eps)
        #T.assemble(K, 'rookPivoting', is_sym, is_pd)

        #Σ12 = kernel_func(X1, X2)
        # Solve
        A12=K.getMatrixEntry(X1,X2)
        
        solved = scipy.linalg.solve(A11, A12, assume_a='pos').T
        # Compute posterior mean
        μ2 = solved @ y1
        
        
        A22=K.getMatrixEntry(X2,X2)
        
        # Compute the posterior covariance
        Σ2 = A22 - (solved @ A12)
        #print("A11 shape",A11.shape)
        #print("A12 shape",A12.shape)
        #print("A22 shape",A22.shape)
        return μ2, Σ2  # mean, covariance
    # Gaussian process posterior
    def GP(self,X1, y1, X2, kernel_func):
        """
        Calculate the posterior mean and covariance matrix for y2
        based on the corresponding input X2, the observations (y1, X1), 
        and the prior kernel function.
        """
        # Kernel of the observations
        Σ11 = kernel_func(X1, X1)
        print("Σ11",Σ11)
        # Kernel of observations vs to-predict
        Σ12 = kernel_func(X1, X2)
        # Solve
        solved = scipy.linalg.solve(Σ11, Σ12, assume_a='pos').T
       
        # Compute posterior mean
        μ2 = solved @ y1

        # Compute the posterior covariance
        Σ22 = kernel_func(X2, X2)
        
        Σ2 = Σ22 - (solved @ Σ12)

        return μ2, Σ2  # mean, covariance
# Define the exponentiated quadratic 
def exponentiated_quadratic(xa, xb):
    """Exponentiated quadratic  with σ=1"""
    # L2 distance (Squared Euclidian)
    sq_norm = -0.5 * dist.cdist(xa, xb, 'sqeuclidean')
    return np.exp(sq_norm)

In [7]:
# Compute the posterior mean and covariance

# Define the true function that we want to regress on
f_sin = lambda x: (np.sin(x)).flatten()

N1 = 6  # Number of points to condition on (training points)
N2 = 5  # Number of points in posterior (test points)
ny = 5  # Number of functions that will be sampled from the posterior
domain = (-6, 6)

# Sample observations (X1, y1) on the function
X1 = np.random.uniform(domain[0]+2, domain[1]-2, size=(N1, 1))
print(X1.shape)
y1 = f_sin(X1)
# Predict points at uniform spacing to capture function
X2 = np.linspace(domain[0], domain[1], N2).reshape(-1, 1)
# Compute posterior mean and covariance
K = Kernel(N1)
μ2_hod, Σ2_hod = K.GP_hod(X1, y1, X2)
print(μ2_hod, Σ2_hod)
print("******")
μ2, Σ2 = K.GP(X1, y1, X2,exponentiated_quadratic)
print(μ2, Σ2)
#print(Σ2.shape)
# Compute the standard deviation at the test points to be plotted
σ2 = np.sqrt(np.diag(Σ2))
σ2_hod = np.sqrt(np.diag(Σ2_hod))

# Draw some samples of the posterior
y2 = np.random.multivariate_normal(mean=μ2, cov=Σ2, size=1)
y2_hod = np.random.multivariate_normal(mean=μ2_hod, cov=Σ2_hod, size=1)

(6, 1)
[-7.85736563e-06 -1.91405856e-01  2.46592580e-02  1.42348404e-01
 -1.13202298e-01] [[1.00000000e+00 1.11069324e-02 1.48201468e-06 4.77285645e-09
  8.46664501e-08]
 [1.11069324e-02 9.52127545e-01 2.43234995e-02 9.39059325e-05
  1.67273822e-03]
 [1.48201468e-06 2.43234995e-02 1.96002712e-02 2.06722188e-04
  4.08941114e-03]
 [4.77285645e-09 9.39059325e-05 2.06722188e-04 1.26966043e-05
  6.44446029e-04]
 [8.46664501e-08 1.67273822e-03 4.08941114e-03 6.44446029e-04
  9.75889108e-01]]
******
[-7.85736563e-06 -1.91405856e-01  2.46592580e-02  1.42348404e-01
 -1.13202298e-01] [[1.00000000e+00 1.11069324e-02 1.48201468e-06 4.77285645e-09
  8.46664501e-08]
 [1.11069324e-02 9.52127545e-01 2.43234995e-02 9.39059325e-05
  1.67273822e-03]
 [1.48201468e-06 2.43234995e-02 1.96002712e-02 2.06722188e-04
  4.08941114e-03]
 [4.77285645e-09 9.39059325e-05 2.06722188e-04 1.26966043e-05
  6.44446029e-04]
 [8.46664501e-08 1.67273822e-03 4.08941114e-03 6.44446029e-04
  9.75889108e-01]]


# Gpr using HODLR properties

In [3]:
import numpy as np
import pyhodlrlib

In [27]:
# Gaussian process posterior
# Returning the Gaussian Kernel:
import scipy
from scipy.spatial import distance as dist
class Kernel1(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
         return np.exp(-0.5*(X1[i] - X1[j])**2)
    def GP_hod(self,X1, y1, X2):
        # What we are doing here is explicitly generating 
        # the matrix from its entries
        A11=K1.getMatrix(0,0,N1,N1)
        #print(A11,"A11")
        return A11
class Kernel2(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
         return np.exp(-0.5*(X1[i] - X2[j])**2)
    def GP_hod(self,X1, y1, X2):
       
        # the matrix from its entries
        A12=K2.getMatrix(0,0,N1,N2)
        #print(A12,"A12")
        
        return A12
class Kernel3(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
         return np.exp(-0.5*(X2[i] - X2[j])**2)
    def GP_hod(self,X1, y1, X2):
        
        # the matrix from its entries
        A22=K3.getMatrix(0,0,N2,N2)
        #print(A22,"A22")

        return A22
# Define the exponentiated quadratic 
def exponentiated_quadratic(xa, xb):
    """Exponentiated quadratic  with σ=1"""
    # L2 distance (Squared Euclidian)
    sq_norm = -0.5 * dist.cdist(xa, xb, 'sqeuclidean')
    return np.exp(sq_norm)
 # Gaussian process posterior
def GP(X1, y1, X2, kernel_func):
    """
    Calculate the posterior mean and covariance matrix for y2
    based on the corresponding input X2, the observations (y1, X1), 
    and the prior kernel function.
    """
    # Kernel of the observations
    Σ11 = kernel_func(X1, X1)
    
    # Kernel of observations vs to-predict
    Σ12 = kernel_func(X1, X2)
    # Solve
    solved = scipy.linalg.solve(Σ11, Σ12, assume_a='pos').T
    print("*****")
    #print("splved",solved)
    # Compute posterior mean
    μ2 = solved @ y1

    # Compute the posterior covariance
    Σ22 = kernel_func(X2, X2)
    Σ2 = Σ22 - (solved @ Σ12)
    #print("Σ11",Σ12,Σ12.shape)
    return μ2, Σ2  # mean, covariance

In [78]:
# Compute the posterior mean and covariance

# Define the true function that we want to regress on
f_sin = lambda x: (np.sin(x)).flatten()

N1 = 6  # Number of points to condition on (training points)
N2 = 5  # Number of points in posterior (test points)
#ny = 5  # Number of functions that will be sampled from the posterior
domain = (-6, 6)

# Sample observations (X1, y1) on the function
X1 = np.random.uniform(domain[0]+2, domain[1]-2, size=(N1, 1))
#print(type(X1))
y1 = f_sin(X1)
# Predict points at uniform spacing to capture function
X2 = np.linspace(domain[0], domain[1], N2).reshape(-1, 1)
#print(X2.shape)
y2_act=f_sin(X2)
# Compute posterior mean and covariance
K1 = Kernel1(N1)
A11 = K1.GP_hod(X1, y1, X1)
K2 = Kernel2(N2)
A12 = K2.GP_hod(X1, y1, X2)
K3 = Kernel3(N2)
A22 = K3.GP_hod(X2, y1, X2)
print("X1,X2",X1.shape,X2.shape)
#print("A11",A12,A12.shape)
eps = 1e-12
# If we are assembling a symmetric matrix:
is_sym = True
# If we know that the matrix is also PD:
# By setting the matrix to be symmetric-positive definite, 
# we trigger the fast symmetric factorization method to be used
# In all other cases the fast factorization method is used
is_pd = True
# Creating the HODLR object:
#print(type(A11))
M=3
T = pyhodlrlib.HODLR(N1, M, eps)
T.assemble(K1, 'rookPivoting', is_sym, is_pd)
# Factorize elements of the tree:
T.factorize()
# Solving for x in A x = b:
x_hodlr = T.solve(A12)
x_hodlr=x_hodlr.T
# Compute posterior mean
#print(x_hodlr.shape,y1.shape,A12.shape,A22.shape)
print("x_hodlr",x_hodlr.shape,'\n',"y1",y1.shape,'\n',"A11",A11.shape,'\n',"A12",A12.shape,'\n',"A22",A22.shape)
μ2_hod = x_hodlr @ y1
Σ2_hod = A22 - (x_hodlr @ A12)
print("mean using HODLR",μ2_hod)
print("covariance matrix using HODLR",Σ2_hod)
print("******")
μ2, Σ2 = GP(X1, y1, X2,exponentiated_quadratic)
print("Mean from scratch",μ2)
print("Covariance matrix from scratch",Σ2)
# Compute the standard deviation at the test points to be plotted
σ2 = np.sqrt(np.diag(Σ2))
σ2_hod = np.sqrt(np.diag(Σ2_hod))

# Draw some samples of the posterior
y2 = np.random.multivariate_normal(mean=μ2, cov=Σ2, size=1)
y2_hod = np.random.multivariate_normal(mean=μ2_hod, cov=Σ2_hod, size=1)

X1,X2 (6, 1) (5, 1)
x_hodlr (5, 6) 
 y1 (6,) 
 A11 (6, 6) 
 A12 (6, 5) 
 A22 (5, 5)
mean using HODLR [ 2.95950151e-04 -2.98028777e-01 -2.53298864e-01  1.41255739e-01
 -8.15246492e-02]
covariance matrix using HODLR [[ 9.99992932e-01  9.03281877e-03 -6.40086471e-04  3.01391907e-09
   6.36767365e-07]
 [ 9.03281877e-03  1.39212250e-01 -7.37882934e-02  4.08709913e-07
   8.67479635e-05]
 [-6.40086471e-04 -7.37882934e-02  3.77289156e-01 -1.48626794e-05
  -3.99473150e-03]
 [ 3.01391907e-09  4.08709913e-07 -1.48626794e-05  3.08174752e-08
   2.96391420e-05]
 [ 6.36767365e-07  8.67479635e-05 -3.99473150e-03  2.96391420e-05
   9.75956148e-01]]
******
*****
Mean from scratch [ 2.95950151e-04 -2.98028777e-01 -2.53298864e-01  1.41255739e-01
 -8.15246492e-02]
Covariance matrix from scratch [[ 9.99992932e-01  9.03281877e-03 -6.40086471e-04  3.01391906e-09
   6.36767365e-07]
 [ 9.03281877e-03  1.39212250e-01 -7.37882934e-02  4.08709913e-07
   8.67479635e-05]
 [-6.40086471e-04 -7.37882934e-02  3.77289156

In [79]:
y2[0]

array([ 1.91950345, -0.03143034,  0.05368037,  0.141281  ,  0.42639043])

In [80]:
y2_hod[0]

array([ 0.08698606, -0.54251342, -0.09087583,  0.14134053,  0.31898221])

In [81]:
(y2_act)

array([ 0.2794155 , -0.14112001,  0.        ,  0.14112001, -0.2794155 ])

In [82]:
x_hodlr

array([[-6.72781252e-03, -2.26017372e-04,  2.59084614e-03,
         2.39385316e-04, -1.81655475e-05,  6.01720781e-03],
       [-1.20887212e+00, -3.08153805e-02,  3.61310547e-01,
         3.26362461e-02, -2.47540427e-03,  1.69563688e+00],
       [-1.18082305e+00,  1.51035440e+00,  1.27309411e+00,
        -1.59274969e+00,  1.16430001e-01,  5.97042700e-01],
       [ 4.87627852e-06,  7.57401086e-01, -3.74201568e-06,
         2.44080464e-01, -1.46734630e-03, -2.75074419e-06],
       [ 1.02720180e-03,  4.06444959e+00, -7.83675337e-04,
        -4.47964136e+00,  5.37447196e-01, -5.80890715e-04]])

In [83]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y2_act,y2_hod[0])
print('MAPE score is {}'.format(mape))

MAPE score is 81853670021895.22


In [84]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y2_act,y2[0])
print('MAPE score is {}'.format(mape))

MAPE score is 48350981259797.336


In [48]:
import pandas as pd

In [65]:
# Gaussian process posterior
# Returning the Gaussian Kernel:
import scipy
from scipy.spatial import distance as dist
class Kernel1(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
        dist=np.linalg.norm(X1[i] - X1[j])
        return np.exp(-0.5*(dist)**2)
         
    def GP_hod(self,X1, y1, X2):
        # What we are doing here is explicitly generating 
        # the matrix from its entries
        A11=K1.getMatrix(0,0,N1,N1)
        #print(A11,"A11")
        return A11
class Kernel2(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
        dist=np.linalg.norm(X1[i] - X2[j])
        return np.exp(-0.5*(dist)**2)
        
    def GP_hod(self,X1, y1, X2):
       
        # the matrix from its entries
        A12=K2.getMatrix(0,0,N1,N2)
        #print(A12,"A12")
        
        return A12
class Kernel3(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
        dist=np.linalg.norm(X2[i] - X2[j])
        return np.exp(-0.5*(dist)**2)
         
    def GP_hod(self,X1, y1, X2):
        
        # the matrix from its entries
        A22=K3.getMatrix(0,0,N2,N2)
        #print(A22,"A22")

        return A22
# Define the exponentiated quadratic 
def exponentiated_quadratic(xa, xb):
    """Exponentiated quadratic  with σ=1"""
    # L2 distance (Squared Euclidian)
    sq_norm = -0.5 * dist.cdist(xa, xb, 'sqeuclidean')
    return np.exp(sq_norm)
def rbf_kernel(X1, X2):
    n_samples_X1, n_features = X1.shape
    n_samples_X2, _ = X2.shape
    kernel_matrix = np.zeros((n_samples_X1, n_samples_X2))

    for i in range(n_samples_X1):
        for j in range(n_samples_X2):
            squared_distance = np.linalg.norm(X1[i] - X2[j])
            kernel_matrix[i, j] = np.exp(-0.5* (squared_distance)**2)

    return kernel_matrix
 # Gaussian process posterior
def GP(X1, y1, X2, kernel_func):
    """
    Calculate the posterior mean and covariance matrix for y2
    based on the corresponding input X2, the observations (y1, X1), 
    and the prior kernel function.
    """
    # Kernel of the observations
    Σ11 = kernel_func(X1, X1)
    
    # Kernel of observations vs to-predict
    Σ12 = kernel_func(X1, X2)
    # Solve
    solved = scipy.linalg.solve(Σ11, Σ12, assume_a='pos').T
    print("*****")
    #print("splved",solved)
    # Compute posterior mean
    μ2 = solved @ y1

    # Compute the posterior covariance
    Σ22 = kernel_func(X2, X2)
    Σ2 = Σ22 - (solved @ Σ12)
    #print("Σ11",Σ12,Σ12.shape)
    return μ2, Σ2  # mean, covariance

In [77]:
# Compute the posterior mean and covariance



# Sample observations (X1, y1) on the function
X1 = np.array([[1,2,3],[2,3,4],[4,5,6]])
#print("X1",X1.shape)#.reshape(-1,1)
y1 = np.array([2,3,4])
#print("y1",y1.shape)

# Predict points at uniform spacing to capture function
X2 = np.array([[5,7,9],[4,8,3]])
print("X1,X2",X1.shape,X2.shape)

N1=len(X1)
N2=len(X2)
# Compute posterior mean and covariance
K1 = Kernel1(N1)
A11 = K1.GP_hod(X1, y1, X1)
#A11=A11.reshape(3,3)
K2 = Kernel2(N2)
A12 = K2.GP_hod(X1, y1, X2)
#A12
K3 = Kernel3(N2)
A22 = K3.GP_hod(X2, y1, X2)

#print("A11",A12,A12.shape)
eps = 1e-12
# If we are assembling a symmetric matrix:
is_sym = True
# If we know that the matrix is also PD:
# By setting the matrix to be symmetric-positive definite, 
# we trigger the fast symmetric factorization method to be used
# In all other cases the fast factorization method is used
is_pd = True
# Creating the HODLR object:
#print(type(A11))
M=3
T = pyhodlrlib.HODLR(N1, M, eps)
T.assemble(K1, 'rookPivoting', is_sym, is_pd)
# Factorize elements of the tree:
T.factorize()
# Solving for x in A x = b:
x_hodlr = T.solve(A12)
x_hodlr=x_hodlr.T
# Compute posterior mean
print("x_hodlr",x_hodlr.shape,'\n',"y1",y1.shape,'\n',"A11",A11.shape,'\n',"A12",A12.shape,'\n',"A22",A22.shape)
μ2_hod = x_hodlr @ y1
Σ2_hod = A22 - (x_hodlr @ A12)
print("mean using HODLR",μ2_hod)
print("covariance matrix using HODLR",Σ2_hod)
print("******")
μ2, Σ2 = GP(X1, y1, X2,rbf_kernel)
print("Mean from scratch",μ2)
print("Covariance matrix from scratch",Σ2)
# Compute the standard deviation at the test points to be plotted
σ2 = np.sqrt(np.diag(Σ2))
σ2_hod = np.sqrt(np.diag(Σ2_hod))

# Draw some samples of the posterior
y2 = np.random.multivariate_normal(mean=μ2, cov=Σ2, size=1)
y2_hod = np.random.multivariate_normal(mean=μ2_hod, cov=Σ2_hod, size=1)

X1,X2 (3, 3) (2, 3)
x_hodlr (2, 3) 
 y1 (3,) 
 A11 (3, 3) 
 A12 (3, 2) 
 A22 (2, 2)
mean using HODLR [0.00364147 0.00049364]
covariance matrix using HODLR [[ 9.99999168e-01 -1.06932378e-07]
 [-1.06932378e-07  9.99999985e-01]]
******
*****
Mean from scratch [0.00364147 0.00049364]
Covariance matrix from scratch [[ 9.99999168e-01 -1.06932378e-07]
 [-1.06932378e-07  9.99999985e-01]]
